In [1]:
import os, json
import pandas as pd
from sqlalchemy import create_engine
from utils.database import process_file, insert_article, delete_articles

<h3>Insert snippets into database</h3>

In [3]:
#set wikipedia folder where dump json files are located
wikifiles = "F:/RESEARCH/TESE/corpora/wikifiles/01012018/json"
os.chdir(wikifiles)

#set conn string for postgresql database
conn_string = "dbname=sparsenlp user=postgres password=xxxxxxx"

#delete all articles from database
delete_articles(conn_string)

#n_articles = 60
n_articles = 10
i = 0
for subdir, dirs, files in os.walk(wikifiles):
    
    for file in files:
        
        if i >= n_articles:
            continue
        elif i % 5 == 0:
            print (str(i)+ ' articles processed')
        #print (os.path.join(subdir, file))
        filepath = subdir + os.sep + file
        
        with open(filepath, 'r', encoding='utf-8') as fh:
            data=fh.readlines()
            process_file(conn_string, data)
        
        i = i + 1
        
print ("DONE. %s articles loaded "% (i))

0 articles processed
5 articles processed
DONE. 10 articles loaded 


<h3>Export snippets to csv file</h3>

In [ ]:
engine = create_engine('postgresql://postgres@localhost:5432/sparsenlp')
text_length = 200
sql = 'select a.id as curid, a.title, lower(b.text) as text from articles a, snippets b '
sql += ' where a.id = b.article_id and length(b.text) > '+str(text_length)
df = pd.read_sql_query(sql, con=engine)

#print (df.shape)
#print (df.head(5))
df.to_csv('F:/RESEARCH/TESE/corpora/wikifiles/snippets.bz2', compression='bz2', encoding='utf-8', header=['curid','title','text'])